In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

In [0]:
import torchvision
import torchvision.transforms as transforms

In [0]:
transform = transforms.Compose([transforms.ToTensor()])

In [4]:
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 26798250.25it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 459189.04it/s]
  1%|          | 16384/1648877 [00:00<00:11, 143676.83it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7549290.99it/s]                            
8192it [00:00, 178674.07it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
class RNN(nn.Module):
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs):
        super(RNN, self).__init__()
        
        self.n_neurons = n_neurons
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        
        self.basic_rnn = nn.RNN(self.n_inputs, self.n_neurons) 
        
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
    def init_hidden(self,):
        return (torch.zeros(1, self.batch_size, self.n_neurons))
        
    def forward(self, X):
        X = X.permute(1, 0, 2) 
        
        self.batch_size = X.size(1)
        self.hidden = self.init_hidden()
        
        lstm_out, self.hidden = self.basic_rnn(X, self.hidden)      
        out = self.FC(self.hidden)
        
        return out.view(-1, self.n_outputs)

In [0]:
import torch.optim as optim

model = RNN(64, 28, 28, 100, 100)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def get_accuracy(logit, target, batch_size):
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [12]:
for epoch in range(20):
    train_running_loss = 0.0
    train_acc = 0.0
    model.train()
    
    for i, data in enumerate(trainloader):
        optimizer.zero_grad()
        
        model.hidden = model.init_hidden() 
        
        inputs, labels = data
        inputs = inputs.view(-1, 28,28) 

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(outputs, labels, 64)
         
    model.eval()
    print('Epoch:  %d , Loss: %.4f , train accuracy: %.2f' %(epoch+1, train_running_loss / i, train_acc/i))

Epoch:  1 , Loss: 1.8728 , train accuracy: 30.81
Epoch:  2 , Loss: 0.6286 , train accuracy: 79.04
Epoch:  3 , Loss: 0.3708 , train accuracy: 88.42
Epoch:  4 , Loss: 0.2820 , train accuracy: 91.31
Epoch:  5 , Loss: 0.2409 , train accuracy: 92.51
Epoch:  6 , Loss: 0.2124 , train accuracy: 93.53
Epoch:  7 , Loss: 0.1988 , train accuracy: 93.76
Epoch:  8 , Loss: 0.1809 , train accuracy: 94.37
Epoch:  9 , Loss: 0.1666 , train accuracy: 94.85
Epoch:  10 , Loss: 0.1567 , train accuracy: 95.07
Epoch:  11 , Loss: 0.1479 , train accuracy: 95.41
Epoch:  12 , Loss: 0.1450 , train accuracy: 95.42
Epoch:  13 , Loss: 0.1397 , train accuracy: 95.58
Epoch:  14 , Loss: 0.1311 , train accuracy: 95.80
Epoch:  15 , Loss: 0.1328 , train accuracy: 95.87
Epoch:  16 , Loss: 0.1370 , train accuracy: 95.71
Epoch:  17 , Loss: 0.1375 , train accuracy: 95.58
Epoch:  18 , Loss: 0.1306 , train accuracy: 95.91
Epoch:  19 , Loss: 0.1166 , train accuracy: 96.26
Epoch:  20 , Loss: 0.1136 , train accuracy: 96.38
